In [1]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 9999))

print("🟢 UDP Chat Server Started on port 9999...")

clients = {}  # client_address -> username
valid_users = {"user1": "pass123", "user2": "mypassword"}  # example credentials

while True:
    data, address = server_socket.recvfrom(1024)
    message = data.decode()

    # Check if client is already authenticated
    if address not in clients:
        try:
            username, password = message.split(",", 1)
        except ValueError:
            server_socket.sendto(b"Invalid format. Use username,password", address)
            continue

        if username in valid_users and valid_users[username] == password:
            clients[address] = username
            server_socket.sendto(b"Authenticated! Welcome to the chat.", address)
            print(f"✅ {username} authenticated from {address}")
        else:
            server_socket.sendto(b"Authentication failed.", address)
        continue

    # Broadcast authenticated client messages
    print(f"📩 Message from {clients[address]} ({address}): {message}")
    for client in clients:
        if client != address:
            server_socket.sendto(f"{clients[address]}: {message}".encode(), client)

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

In [ ]:
def encrypt(message, key=3):
    result = ""
    for char in message:
        if char.isalpha():
            shift = key
            if char.islower():
                result += chr((ord(char) - ord('a') + shift) % 26 + ord('a'))
            else:
                result += chr((ord(char) - ord('A') + shift) % 26 + ord('A'))
        else:
            result += char
    return result

def decrypt(message, key=3):
    return encrypt(message, -key)  # decrypt by shifting backward

Enter username:  pass123
Enter password:  Unty456#


In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 9999))

print("🟢 UDP Chat Server Started on port 9999...")

clients = {}  # client_address -> username
valid_users = {"user1": "pass123", "user2": "mypassword"}  # example credentials

while True:
    data, address = server_socket.recvfrom(1024)
    message = data.decode()

    # Check if client is already authenticated
    if address not in clients:
        try:
            username, password = message.split(",", 1)
        except ValueError:
            server_socket.sendto(b"❌ Invalid format. Use username,password", address)
            continue

        if username in valid_users and valid_users[username] == password:
            clients[address] = username
            server_socket.sendto(b"✅ Authenticated! Welcome to the chat.", address)
            print(f"✅ {username} authenticated from {address}")
        else:
            server_socket.sendto(b"❌ Authentication failed.", address)
        continue

    # Broadcast authenticated client messages
    print(f"📩 Message from {clients[address]} ({address}): {message}")
    for client in clients:
        if client != address:
            server_socket.sendto(f"{clients[address]}: {message}".encode(), client)

 # Inside server message broadcast
decrypted_msg = decrypt(message)
print(f"📩 Message from {clients[address]} ({address}): {decrypted_msg}")

for client in clients:
    if client != address:
        # Re-encrypt before sending
        server_socket.sendto(encrypt(f"{clients[address]}: {decrypted_msg}").encode(), client)

In [ ]:
import socket
import threading

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 9999)

# Enter credentials
username = input("Enter username: ")
password = input("Enter password: ")
auth_message = f"{username},{password}"
client_socket.sendto(auth_message.encode(), server_address)

# Receive authentication response
data, _ = client_socket.recvfrom(1024)
print(data.decode())
if b"❌" in data:
    exit()  # stop if authentication failed

# Function to receive messages
def receive_messages():
    while True:
        data, _ = client_socket.recvfrom(1024)
        print("\n💬", data.decode())

threading.Thread(target=receive_messages, daemon=True).start()

while True:
    message = input()
    encrypted_msg = encrypt(message)
    client_socket.sendto(encrypted_msg.encode(), server_address)